In [100]:
import pickle as pkl
from dynGENIE3 import * #Or wherever you locate dynGENIE3 in.
import pandas as pd
import numpy  as np
import networkx as nx
import matplotlib.pyplot as plt
import timeit
%matplotlib inline

In [101]:
cs_data = pd.read_csv('computer_science_conference_subset.csv' ,header = 0, \
                      names=['article','topic','conference','year','date'], index_col = 0)
cs_data.drop(columns=['date'],inplace=True)
cs_data.sort_values(by='year',inplace=True)

In [102]:
cs_data.shape

(4127678, 4)

In [103]:
cs_data.head()

,article,topic,conference,year
3357460,2621048332,23123220,1.133524e+09,1800.0
3357462,2621048332,44291984,1.133524e+09,1800.0
3357461,2621048332,41008148,1.133524e+09,1800.0
3357459,2621048332,77088390,1.133524e+09,1800.0
3305368,2565600429,16005928,1.130177e+09,1916.0


In [104]:
picked_keywords = pd.read_csv('Computer_Science_Conference_Subset_Top_Keywords_JunpeiPickUp.csv', index_col = 0)

In [105]:
cs_data = cs_data[cs_data['topic'].isin(picked_keywords['topic_30_years'])]

In [106]:
##Remain recent years and somewhat frequent journal/topic.
cs_data = cs_data[(cs_data.year>=1990) & (cs_data.year<=2019)]
conference_list = cs_data['conference'].unique()
# conference_count = cs_data.groupby('conference').count().reset_index()
# conference_list = conference_count.conference[  conference_count.article>=1000].unique()
# topic_count = cs_data.groupby('topic').count().reset_index()
# topic_list = topic_count.topic[  topic_count.article>=500].unique()
topic_list = cs_data['topic'].unique()
year_list = cs_data.year.unique()

# cs_data = cs_data[cs_data.conference.isin(conference_list)]  
# cs_data = cs_data[cs_data.topic.isin(topic_list)]  

In [107]:
cs_data['topic'].nunique()

190

In [108]:
cs_data

,article,topic,conference,year
1375444,2073039908,76482347,1.169675e+09,1990.0
374734,1563621278,144171764,1.163902e+09,1990.0
1375445,2073039908,60048249,1.169675e+09,1990.0
2219142,2125525147,106516650,1.143724e+09,1990.0
2706241,2158348710,144171764,1.143279e+09,1990.0
...,...,...,...,...
3655038,2907063161,185429906,2.622963e+09,2019.0
3901664,2954907277,76449508,1.140000e+09,2019.0
3901723,2962919941,52102323,1.158168e+09,2019.0
3655044,2907063161,97541855,2.622963e+09,2019.0


In [109]:
##Frequency 
cs_count = cs_data.groupby(['topic','conference','year']).count() 
cs_count.rename( columns={'article':'count'},inplace=True )
cs_count.reset_index(inplace=True)

In [110]:
cs_count

,topic,conference,year,count
0,58166,1.121228e+09,1990.0,2
1,58166,1.121228e+09,1992.0,3
2,58166,1.121228e+09,1993.0,3
3,58166,1.121228e+09,1994.0,4
4,58166,1.121228e+09,1995.0,3
...,...,...,...,...
86841,2908647359,2.623113e+09,2016.0,13
86842,2908647359,2.623113e+09,2017.0,19
86843,2908647359,2.623113e+09,2018.0,33
86844,2908647359,2.623113e+09,2019.0,14


In [111]:
##Create time-series for dyGENIE3.
cs_wide = cs_count.pivot_table(index=['year','topic'], columns='conference', values='count')
cs_wide.fillna(0, inplace=True)
cs_wide.sort_values(by='year',inplace=True)
cs_wide.reset_index(inplace=True)
TS_data, time_points, decay_rates, gene_names = [], [], [], []
decay_rates = np.zeros( len(conference_list) )
gene_names = conference_list

In [112]:
for topic in topic_list:
    data = cs_wide[cs_wide['topic']==topic]
    TS_data.append( data.drop(columns=['year', 'topic']).to_numpy() )
    time_points.append( data['year'].to_numpy() )

In [113]:
(VIM, alphas, prediction_score,stability_score,treeEstimators) = dynGENIE3(TS_data, time_points,alpha=decay_rates)

Tree method: RF
K: sqrt
Number of trees: 1000
alpha min: 0.0
alpha max: 0.0


running single threaded jobs
Gene 1/62...
Gene 2/62...
Gene 3/62...
Gene 4/62...
Gene 5/62...
Gene 6/62...
Gene 7/62...
Gene 8/62...
Gene 9/62...
Gene 10/62...
Gene 11/62...
Gene 12/62...
Gene 13/62...
Gene 14/62...
Gene 15/62...
Gene 16/62...
Gene 17/62...
Gene 18/62...
Gene 19/62...
Gene 20/62...
Gene 21/62...
Gene 22/62...
Gene 23/62...
Gene 24/62...
Gene 25/62...
Gene 26/62...
Gene 27/62...
Gene 28/62...
Gene 29/62...
Gene 30/62...
Gene 31/62...
Gene 32/62...
Gene 33/62...
Gene 34/62...
Gene 35/62...
Gene 36/62...
Gene 37/62...
Gene 38/62...
Gene 39/62...
Gene 40/62...
Gene 41/62...
Gene 42/62...
Gene 43/62...
Gene 44/62...
Gene 45/62...
Gene 46/62...
Gene 47/62...
Gene 48/62...
Gene 49/62...
Gene 50/62...
Gene 51/62...
Gene 52/62...
Gene 53/62...
Gene 54/62...
Gene 55/62...
Gene 56/62...
Gene 57/62...
Gene 58/62...
Gene 59/62...
Gene 60/62...
Gene 61/62...
Gene 62/62...
Elapsed time: 168.75 seconds


In [ ]:
VIM.to_csv('journal_network_VIM.csv')

In [ ]:
# VIM = pd.read_csv('journal_network_VIM.csv')

In [114]:
result=pd.DataFrame(VIM,index=gene_names,columns=gene_names)  

In [115]:
conference_name = pd.read_csv('./All data/mag/ConferenceSeries.txt', delimiter = "\t", header = None,
                              names = ['ConferenceSeriesId', 'Rank', 'NormalizedName', 'DisplayName', 'PaperCount',\
                                       'CitationCount', 'CreatedDate'])
conference_subset_name = conference_name[conference_name['ConferenceSeriesId'].isin(result.index)]

In [ ]:
result_withname = result.reset_index().merge(conference_subset_name[['ConferenceSeriesId', 'DisplayName']],\
                                             left_on = 'index', right_on = 'ConferenceSeriesId')


In [ ]:
result_withname.drop(0, axis = 1, inplace = True)

In [ ]:
result_withname.to_csv('journal_network.csv')

In [116]:
g=nx.from_numpy_array(VIM, create_using=nx.DiGraph) 

## Computer HITS Ranking

In [117]:
h,a=nx.hits(g)

In [118]:
result_df = pd.DataFrame( index=result.index  )
result_df['hub_index'] = h.values()
result_df['authority_index'] = a.values()

In [119]:
result_df

,hub_index,authority_index
1.169675e+09,0.009405,0.015018
1.163902e+09,0.039590,0.016964
1.143724e+09,0.010659,0.015970
1.143279e+09,0.019593,0.016589
1.153524e+09,0.009411,0.013821
...,...,...
1.173394e+09,0.010297,0.016660
1.134284e+09,0.006238,0.013254
1.120384e+09,0.007933,0.016086
2.754508e+09,0.016449,0.015871


In [120]:
# Add journal names

result_hits_withname = result_df.reset_index().merge(conference_subset_name[['ConferenceSeriesId', 'DisplayName', 'NormalizedName']],\
                                                     left_on = 'index', right_on = 'ConferenceSeriesId')

In [121]:
result_hits_withname.drop('index', axis = 1, inplace = True)
result_hits_withname.sort_values('authority_index', inplace = True, ascending = False)

In [122]:
result_hits_withname

,hub_index,authority_index,ConferenceSeriesId,DisplayName,NormalizedName
17,0.027573,0.019594,1124077590,European Conference on Computer Vision,ECCV
39,0.019134,0.018979,1185109434,Operating Systems Design and Implementation,OSDI
55,0.022068,0.018912,1183478919,International Conference on Data Mining,ICDM
43,0.012868,0.018790,2622962978,International Conference on Artificial Intelli...,AISTATS
14,0.029703,0.018481,1183230087,International Solid-State Circuits Conference,ISSCC
...,...,...,...,...,...
46,0.006566,0.014155,1192655580,Empirical Methods in Natural Language Processing,EMNLP
4,0.009411,0.013821,1153524033,International Cryptology Conference,CRYPTO
22,0.010962,0.013511,1184914352,National Conference on Artificial Intelligence,AAAI
12,0.007057,0.013508,1131341566,International Symposium on Computer Architecture,ISCA


In [123]:
result_hits_withname.to_csv('cs_conference_hits.csv')

## Visualization

In [ ]:
edge_200th_index = VIM.ravel().argsort()[-200:][0]
edge_100th_index = VIM.ravel().argsort()[-100:][0]
edge_50th_index = VIM.ravel().argsort()[-50:][0]
edge_25th_index = VIM.ravel().argsort()[-25:][0]

In [ ]:
edge_200th = VIM.ravel()[edge_200th_index]
edge_100th = VIM.ravel()[edge_100th_index]
edge_50th = VIM.ravel()[edge_50th_index]
edge_25th = VIM.ravel()[edge_25th_index]

In [ ]:
# Remove edges below threshold

small_edge = []
for u, v in g.edges():
    if g[u][v]['weight'] < edge_200th:
        small_edge.append((u,v))
g.remove_edges_from(small_edge)

In [ ]:
# Edge color top 25th:black, 25th-100th: grey, 100th-200th: silver

edge_color = []

for u, v in g.edges():
    if g[u][v]['weight'] < edge_100th:
        edge_color.append('silver')
    elif g[u][v]['weight'] >= edge_100th and g[u][v]['weight'] < edge_25th:
        edge_color.append('grey')   
    else:
        edge_color.append('black')        


In [ ]:
# Topic modeling on nodes

from sklearn.decomposition import LatentDirichletAllocation
arch_cluster = arch_data.groupby(['topic','journal']).count() 
arch_cluster.reset_index(inplace=True)

#Each row is a journal, each column is a keyword
arch_cluster = arch_cluster.pivot_table(index=['journal'], columns='topic', values='article')  

arch_cluster.fillna(0, inplace = True)

lda = LatentDirichletAllocation(n_components=3)
RGB = lda.fit_transform(arch_cluster)
RGB_cluster = RGB.argsort(axis = 1)[:,-1]

In [ ]:
# Node color cluster 0: red, cluster 1: green, cluster 2: blue

# node_color = []

# for node in RGB_cluster:
#     if node == 0:
#         node_color.append('red')
#     elif node == 1:
#         node_color.append('green')
#     else:
#         node_color.append('blue')

In [ ]:
node_label = {}
for i, node_index in enumerate(result.index):
    node_label[i] = (journal_subset_name[journal_subset_name[0] == node_index]['Name'].values[0])

In [ ]:
plt.figure(figsize = (20,20))
weights = [g[u][v]['weight'] * 50 for u,v in g.edges()]
pos = nx.spring_layout(g, seed = 100)
# nx.draw_networkx_labels(g, pos = pos, font_size = 8, with_labels = True, labels = node_label, font_color = 'red')

node_20th_index = result_df['authority_index'].ravel().argsort()[-20:][0]

for i in range(len(node_label)):
    if result_df['authority_index'].iloc[i] >= result_df['authority_index'].iloc[node_20th_index]:
        plt.plot([], [], color = RGB[i], label = node_label[i], marker = 'o',\
                 markersize = result_df['authority_index'].iloc[i] * 500, linewidth=0.0)

nx.drawing.nx_pylab.draw_networkx(g, node_color = RGB, edge_color = edge_color, with_labels = False,\
                         font_color = 'white', pos = pos, width = weights, arrowsize = 10, arrow = True,\
                         node_size = result_df['authority_index'].values * 20000)

plt.legend(bbox_to_anchor=(0.5, -0.05), loc = 'center', ncol = 4)
